In [2]:
%matplotlib inline
import numpy as np
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import math
import pickle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import make_scorer
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import seaborn as sns 

sns.set(style="white")

/home/mickael/.local/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [15]:
# utilities
def confusion_matrix_rate(y_true, y_pred_soft, proba_limit = 0.5):
    # We assume that first column of ypred represents the 0 proba 
    # And the second column represent the 1 proba 
    proba_0 = ypred_proba[:,0]
    ypred_reco = [0 if x>proba_limit else 1 for x in proba_0]
    ypred_reco = np.asarray(ypred_reco)
    df = pd.DataFrame(confusion_matrix(ytrue,ypred_reco), 
             columns = ["Predict 0", "Predict 1"], 
             index = ["Real 0", "Real 1"])
    mask_predict_0 = (ypred_reco==0)
    n = len(ytrue)
    num_pred0 = sum(ypred_reco==0)
    num_true0_onpred0 = sum(ytrue[mask_predict_0]==0)
    print("Precision of 0 in our dataset is {}".format(num_true0_onpred0/num_pred0))
    print("For this precision level, we predict {}% of the dataset".format(num_pred0/n *100))
    print(df)

# Load of Data

In [8]:
# Load Training Data
# nrows_load = 50000 
X_train_raw = pd.read_csv('InputData/xtrain_challenge.csv')
y_train_raw = pd.read_csv('InputData/ytrain_challenge.csv')
y_train_raw = np.array(y_train_raw).ravel()

# Create 1st classifier for prediction 0
> The goal of this first classifier is to predict the 0 (unmatch) with the best accuracy. Our goal is to maximize the number of 0 prediction with the most True Positive Rate and the minimun false rate

In [5]:
def scoringStage1(y_true,y_pred):
    conf = confusion_matrix(y_true,y_pred)
    n0_true = conf[0,0]
    n0 = np.sum(conf[:,0])
    n = len(y_true)
    PredictPercentage = n0/n
    PredictPrecision = n0_true/n0
    return PredictPercentage * (PredictPrecision ** 2)
scoring = make_scorer(scoringStage1)

In [6]:
xgb0 = XGBClassifier(learning_rate = 0.05, n_estimators=1000, 
                    objective='binary:logistic', max_depth=4,
                    silent=True)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_train_raw, y_train_raw, test_size=0.20, random_state=42)

In [11]:
cv_results = cross_validate(xgb0, X_train_raw[0:10], y_train_raw[0:10], scoring=scoring, cv=5)

xgb0.fit(X_train[0:10],y_train[0:10])

/home/mickael/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [14]:
cv_results
soft = xgb0.predict_proba(X_test)

In [16]:
confusion_matrix_rate(y_test,soft,0.5)

Precision of 0 in our dataset is 0.9447895100069013
For this precision level, we predict 73.43674965938935% of the dataset
        Predict 0  Predict 1
Real 0     443556      12754
Real 1      25920     157063


# Hyper Tuning 

In [ ]:
# Xgboost parameter
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        'learning_rate': [0.05, 0.1, 0.15, 0.3, 0.5]
        }

In [62]:
cv_results

/home/mickael/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([0.04671049, 0.04121351, 0.04036546, 0.03984928, 0.04319882]),
 'score_time': array([0.00115585, 0.00113678, 0.00115323, 0.00114107, 0.00145698]),
 'test_score': array([0.44444444, 1.        , 1.        , 1.        , 1.        ]),
 'train_score': array([1.        , 0.765625  , 0.765625  , 0.765625  , 0.79012346])}

# Save the model

In [7]:
pkl_filename = "xgboost_stage1_model.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(xgb0, file)